![Gambar Avengers](Avengers.jpg)
# Hybrid Recommendation
###  Oleh :Topik Zulkarnain

Notebook ini bertujuan untuk melakukan suatu rekomendasi dengan menggunakan Hybrid Recommendation System yaitu menggunakan collaborative filtering dan content base terhadap data hasil kuisioner peserta ASTRA Data Scientist Bootcamp. Data kuisioner berupa rating film dan konten berupa genre film yang diberikan oleh para peserta data scientist dan dari data tersebut akan dijadikan bahan rekomendasi film apa yang harus direkomendasikan berdasarkan rating dan genre yang telah diberikan.
***

# Preparation
## Menyiapkan Data

In [101]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn import preprocessing


In [102]:
film_data = pd.read_csv('movies.csv')
rating_data = pd.read_csv('rating.csv')

In [103]:
film_data

,MovieID,Title,Genre
0,1,Ada Apa dengan Cinta 2,Drama
1,2,Gundala,Action
2,3,Dilan 1991,Drama
3,4,Bumi Manusia,Drama
4,5,Dua Garis Biru,Drama
5,6,Avengers: End Game,Action
6,7,The Lion King,Animasi
7,8,Aladdin,Animasi
8,9,Spiderman: Far From Home,Action
9,10,Captain Marvel,Action


In [104]:
rating_data.head()

,Nama,MovieID,Rating
0,Hania,1,3
1,Hania,2,5
2,Hania,3,4
3,Hania,4,4
4,Hania,5,4


data yang digunakan adalah data film yang berisi feature MovieID dan title, sedangkan data rating berisi nama pemberi rating, movie id dan juga rating yang diberikan pada tiap film yang pernah ditonton. Berdasarkan data inilah akan dibuat suatu system rekomendasi film untuk seseorang berdasarkan kesamaannya dalam memberikan rating dan juga konten genre film
dengan menggunakan collaborative filtering dan juga content base. Berbeda dengan collaborative yang hanya menggunakan data rating, untuk hybri reccomendation akan digunakan konten dari film tersebut yang berupa genre film. Oleh karena itu, data genre tidak akan didrop kali ini dan akan digunakan sebagai bahan analisis. Untuk data kategori genre untuk bisa dianalisis haruslah berbentuk numerik, maka dari itu akan digunakan teknik label encoder untuk merubah genre menjadi numerik


In [105]:
moviesWithGenres_df = film_data.copy()

In [106]:
le = preprocessing.LabelEncoder()

In [107]:
le.fit(moviesWithGenres_df['Genre'])

LabelEncoder()

In [108]:
moviesWithGenres_df['Genre'] = le.transform(moviesWithGenres_df['Genre'])

In [109]:
moviesWithGenres_df

,MovieID,Title,Genre
0,1,Ada Apa dengan Cinta 2,2
1,2,Gundala,0
2,3,Dilan 1991,2
3,4,Bumi Manusia,2
4,5,Dua Garis Biru,2
5,6,Avengers: End Game,0
6,7,The Lion King,1
7,8,Aladdin,1
8,9,Spiderman: Far From Home,0
9,10,Captain Marvel,0


## Hybrid : CF+CB
Proses pembuatan sistem rekomendasi film adalah sebagai berikut:
- Memilih nama orang yang sama sama sudah menonton film.
- berdasarkan rating dan genre yang diberikan, akan dicari tetangga yang paling sering menonton film.
- Menggambil record dari orang yang sudah menonton film.
- Menghitung bobot kemiripan antara genre dan rating.
- Rekomendasikan film berdasarkan skor kemiripan yang tertinggi.

kemudian, misalkan ada seorang yang bernama Agung, maka dapat direkomendasikan kepada Agung film apa saja yang dia tonton berdasarkan kemiripan pemberian rating Agung terhadap data yang ada.

In [39]:
Agung_Input = [
            {'Title':'Avengers: End Game', 'Rating':4},
            {'Title':'Gundala', 'Rating':5},
            {'Title':'Dilan 1991', 'Rating':5}
         ] 
inputMovies = pd.DataFrame(Agung_Input)
inputMovies

,Rating,Title
0,4,Avengers: End Game
1,5,Gundala
2,5,Dilan 1991


Data Agung harus terlebih dahulu dimasukan feature MovieID agar nanti bisa diproses untuk mendapatkan rekomendasi film

In [110]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['Title'].isin(inputMovies['Title'].tolist())]
userMovies

,MovieID,Title,Genre
1,2,Gundala,0
2,3,Dilan 1991,2
5,6,Avengers: End Game,0


data diatas adalah data nama orang yang telah menonton dan memberikan rating film dan genre yang diinput Agung

In [111]:
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop('MovieID', 1).drop('Title', 1)
userGenreTable

,Genre
0,0
1,2
2,0


In [113]:
inputMovies['Rating']

0    4
1    5
2    5
Name: Rating, dtype: int64

In [115]:
userProfile = userGenreTable.transpose().dot(inputMovies['Rating'])
userProfile

Genre    10
dtype: int64

In [116]:
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['MovieID'])
genreTable = genreTable.drop('MovieID', 1).drop('Title', 1)
genreTable.head()

,Genre
MovieID,
1,2
2,0
3,2
4,2
5,2


In [123]:
#mengalikan genre dengan bobot
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

MovieID
1    2.0
2    0.0
3    2.0
4    2.0
5    2.0
dtype: float64

In [118]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

MovieID
5    2.0
4    2.0
3    2.0
1    2.0
8    1.0
dtype: float64

In [122]:
#Top 5 film rekomendasi
film_data.loc[film_data['MovieID'].isin(recommendationTable_df.head(5).keys())]

,MovieID,Title,Genre
0,1,Ada Apa dengan Cinta 2,Drama
2,3,Dilan 1991,Drama
3,4,Bumi Manusia,Drama
4,5,Dua Garis Biru,Drama
7,8,Aladdin,Animasi


# Kesimpulan
Film yang direkomendasikan untuk Agung berdasarkan rating dan konten dari genrenya adalah Ada apa dengan Cinta 2, Bumi Manusia, Dua Garis Biru, dan Aladdin